In [1]:
import pandas as pd
import time

start = time.time()

input_file_name = "зц.xlsx"

output_file_name = "costs_cluster_20240411_FF_SFO+UFO_15.csv"


pp = pd.read_excel(input_file_name, dtype="str")


pp.iloc[:, 0] = "0" * 12 + pp.iloc[:, 0].str[-6:]


pp.to_csv(output_file_name, index=False, sep=";")
print ('создан файл - ' + str(time.time() - start))

создан файл - 1.4104835987091064


In [2]:
import pandas as pd
import time

start = time.time()

input_file_name = "new_competitors_2024-04-10-22-47_FF_SFO+UFO_15.xlsx"

output_file_name = "скр пр.xlsx"


pp_file_name = "зц.xlsx"


str_columns = ["Код товара", "ID задачи на мониторинг", "ID конкурента"]

mf = pd.read_excel(input_file_name,
                   dtype={col: str for col in str_columns})

mf["Код товара"] = mf["Код товара"].astype(str)

pp = pd.read_excel(pp_file_name)

tk = pd.read_excel("(правила).xlsx",
                   sheet_name="тк",
                   dtype="str")


pp["material_id"] = pp["material_id"].astype(str)
pp["cluster_id"] = pp["cluster_id"].astype(str)



hp_check = mf[["ID задачи на мониторинг", "Код товара","Внешний код товара", "ID конкурента", 
               "Кластер", "Акционная цена Ленты", "Регулярная цена конкурента", "EPP"]]

hp_check = hp_check[hp_check["Регулярная цена конкурента"].notna()]
hp_check = hp_check.drop_duplicates(subset=["Код товара","Кластер"])


hp = (hp_check[["Код товара", "Кластер", "Регулярная цена конкурента"]]
       .groupby(["Код товара", "Кластер"])["Регулярная цена конкурента"].min().reset_index())




hp = hp.merge(tk[["номер кластера", "кластер"]], how='left', left_on="Кластер", right_on="кластер")
hp = hp.drop_duplicates()


hp = hp.merge(pp, how='left', left_on=["Код товара", "номер кластера"], right_on=["material_id", "cluster_id"])


hp["сп"] = hp.apply(lambda row: "да" if row["Регулярная цена конкурента"] < row["basic_price_amount"]
                   else "нет", axis=1)

hp = hp.merge(hp_check [["Код товара", "Кластер", "Регулярная цена конкурента", "Акционная цена Ленты",
                         "ID задачи на мониторинг", "ID конкурента", "EPP"]], 
              how='left', left_on=["Код товара", "Кластер", "Регулярная цена конкурента"], 
              right_on=["Код товара", "Кластер", "Регулярная цена конкурента"])


hp.to_excel("скр пр.xlsx", index=False)
print ('создан файл - ' + str(time.time() - start))

создан файл - 3.2342333793640137
